# Quick Regression Model on Hacker News Dataset
- Predicting upvotes by using the bag of words model with post title

In [1]:
#analytics
import pandas as pd
import numpy as np

#Natural language processing: Next we import packages to convert the json data from json file and stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

##### Reading in Our Dataset
The dataset has over 200K post titles with the respective number of likes (or number of points) and the number of comments. Since its not possible to train a dataset this huge on the local computer, we're going to have to subset the data

[dataset](https://www.kaggle.com/hacker-news/hacker-news-posts)

In [19]:
hack_data = pd.read_csv('C:/Users/Darshil/gitly/Deep-Learning/My Projects/Flask-app/project_specific/E2_predict_likes_hackernews/hn_posts.csv')
hack_data.head(10)

,title,num_points,num_comments
0,You have two days to comment if you want stem ...,1,0
1,SQLAR the SQLite Archiver,1,0
2,What if we just printed a flatscreen televisio...,1,0
3,algorithmic music,1,0
4,How the Data Vault Enables the Next-Gen Data W...,1,0
5,Saving the Hassle of Shopping,1,1
6,Macalifa A new open-source music app for UWP ...,1,0
7,GitHub theweavrs/Macalifa: A music player wri...,1,0
8,Google Allo first Impression,3,0
9,Advanced Multimedia on the Linux Command Line,1,0


In [45]:
hack_data[hack_data['num_points']>100]

,title,num_points,num_comments
67,Appropriate Uses for SQLite,125,56
88,UnGoogled Chromium: Chromium with enhanced pri...,251,120
127,Designing and producing 2FA tokens to sell on ...,138,52
158,Finance is Not the Economy,237,115
239,The decline of Stack Overflow (2015),291,255
240,Bidirectional Replication is coming to Postgre...,200,38
275,Bay Area wages soaring but still cant keep up ...,103,166
282,Park.io automating tasks to make $125k per month,342,146
288,Game Genie declassified: That summer I played ...,156,40
295,John Carmack .plan Archive (2014),122,38


#### Before we extract our features we need to subset the data

In [22]:
#for now we'll take the first 1000 and then implement later. Idea: concat 700 above 100 and 300 below
df = hack_data.head(1000)

#### Extracting our features

In [23]:
"""
Now we'll apply the bag of words model on our title column bit before we do that we need to clean our data: 
- remove all stop words
- remove words with length < 4
- remove punctuations
"""

# We will now begin cleaning up our 2 columns namely "headline" and "short_description"
punctuations_list = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")", "/"]
sw = stopwords.words('english')

#Here we have created a function that will clean up any given pandases series
def preprocess_textcol(name):
    df['new_{}'.format(name)] = df[name].apply(lambda x: ' '.join([word.strip() for word in x.split() if word not in (sw)
                                                                             and word.lower() not in (sw) 
                                                                             and word.upper() not in sw
                                                                             and word.title() not in (sw)
                                                                         and word not in (punctuations_list) and len(word)>4 
                                                                        and word.isdigit() == False ]))  #numbers doesnt work    
    return df['new_{}'.format(name)] 

#creating 2 new columns out of the original columns "headlines" and "short_description"
df['new_title'] = preprocess_textcol('title')

C:\Users\Darshil\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Darshil\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
#lets view our data
df.head(15)

,title,num_points,num_comments,new_title
0,You have two days to comment if you want stem ...,1,0,comment cells classified
1,SQLAR the SQLite Archiver,1,0,SQLAR SQLite Archiver
2,What if we just printed a flatscreen televisio...,1,0,printed flatscreen television boxes?
3,algorithmic music,1,0,algorithmic music
4,How the Data Vault Enables the Next-Gen Data W...,1,0,Vault Enables Next-Gen Warehouse
5,Saving the Hassle of Shopping,1,1,Saving Hassle Shopping
6,Macalifa A new open-source music app for UWP ...,1,0,Macalifa open-source music
7,GitHub theweavrs/Macalifa: A music player wri...,1,0,GitHub theweavrs/Macalifa: music player written
8,Google Allo first Impression,3,0,Google first Impression
9,Advanced Multimedia on the Linux Command Line,1,0,Advanced Multimedia Linux Command


#### Next we need to create our bag of words model and create a count vector matrix

In [14]:
data_corpus = list(df['title'])
X = vectorizer.fit_transform(data_corpus) 
print(X.toarray())
print(vectorizer.get_feature_names())

test_df = pd.DataFrame(X.toarray(), columns =vectorizer.get_feature_names() )
test_df

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['000', '01', '10', '100k', '11', '12', '125k', '16th', '17', '1778', '1959', '1967', '1978', '1979', '1985', '1993', '1996', '1997', '1999', '2000', '2003', '2007', '2008', '2009', '200mb', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '21', '285m', '29', '35a', '400', '500k', '60fps', '62', 'abruptly', 'absolutely', 'abuse', 'academy', 'accelerated', 'accelerators', 'accenture', 'access', 'accessbility', 'accessible', 'accident', 'accidentally', 'accordions', 'account', 'accounts', 'accreditor', 'acquisition', 'across', 'acting', 'action', 'actions', 'activity', 'actor', 'actors', 'actually', 'adapted', 'address', 'administrator', 'adobe', 'adresses', 'ads', 'advanced', 'advances', 'advice', 'advisory', 'affair', 'affordable', 'africa', 'african', 'agents', 'agree', 'agricultural', 'ahead', 'ai', 'aided', 'airbnb', 'aircraft', 'airlines', 'airpods', 'airpo

,000,01,10,100k,11,12,125k,16th,17,1778,...,years,yelp,yorks,young,youtube,youve,yukon,zeros,zombie,zuboff
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Last but not the least we need to train it using scikit learn 

In [25]:
from sklearn import linear_model

#call model, fit and train
model = linear_model.LinearRegression()
model.fit(test_df,df['num_points'])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
# Now we predict!

test_val = vectorizer.transform(['Google first Impression printed flatscreen television boxes? local development?']) 
print ('New value: ', test_val.toarray()[0])
model.predict(test_val)

New value:  [0 0 0 ... 0 0 0]


array([-22.10218896])

#### This model is horrible we will need to train it on a much larger dataset
#### Next we need to save this model to deploy 

In [39]:
from sklearn.externals import joblib
joblib.dump(model,'predict_likes_hackernews')

PermissionError: [Errno 13] Permission denied: 'C:/Users/Darshil/gitly/Deep-Learning/My Projects/Flask-app/project_specific/E2_predict_likes_hackernews'

In [ ]:
"""# Example code for count vectorization (from other project)
data_corpus = ["John likes to watch movies. Mary likes movies too.", 'darshil also likes to watch movies']
X = vectorizer.fit_transform(data_corpus) 
print(X.toarray())
print(vectorizer.get_feature_names())

test_df = pd.DataFrame(X.toarray(), columns =vectorizer.get_feature_names() )
test_df

#new values
new_val = vectorizer.transform(['hello likes to']) 
print ('New value: ', new_val.toarray()[0])"""